# 🤖 Notebook 2: Criando Agentes Inteligentes com Azure AI

## 📋 O que você vai aprender neste notebook:

1. ✅ O que são agentes de IA e como funcionam
2. ✅ Como criar agentes especializados (matemática, viagens, receitas)
3. ✅ Como dar "ferramentas" (tools) aos agentes
4. ✅ Como criar um orquestrador que gerencia múltiplos agentes
5. ✅ Programação assíncrona (async/await) de forma simples

## ⏱️ Tempo estimado: 40-50 minutos

---

## 🎯 Objetivos de Aprendizado

Ao final deste notebook, você será capaz de:
- Entender o conceito de agentes especializados
- Criar agentes com personalidades e habilidades únicas
- Adicionar ferramentas que os agentes podem usar
- Orquestrar múltiplos agentes para trabalharem juntos

---

## 🤔 O que é um Agente de IA?

Um **agente** é como um assistente especializado que:
- 🎯 Tem uma **função específica** (ex: especialista em matemática, guia turístico)
- 🧠 Pode **usar ferramentas** para realizar tarefas
- 💬 Segue **instruções personalizadas** que você define
- 🔄 Pode **trabalhar com outros agentes** para resolver problemas complexos

**Exemplo do mundo real:**
Imagine uma empresa com diferentes departamentos:
- 🧮 Departamento de Contabilidade (agente de matemática)
- ✈️ Departamento de Viagens (agente de turismo)
- 🎯 Gerente Geral (orquestrador que direciona as perguntas)

---

## 🔧 Tecnologias que vamos usar:

- **Microsoft Agent Framework**: Framework para criar e gerenciar agentes
- **Azure OpenAI**: Os modelos de IA que alimentam os agentes
- **Programação Assíncrona**: Para executar múltiplos agentes eficientemente

---

## 🔑 Pré-requisitos

Certifique-se de que você:
- ✓ Completou o Notebook 1 (chat.ipynb)
- ✓ Tem suas credenciais no arquivo `.env`
- ✓ Instalou todas as dependências

**Se ainda não instalou as dependências:**
```bash
pip install azure-ai-projects azure-identity agent-framework python-dotenv
```

---

In [2]:
import os
import asyncio
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from azure.ai.projects.aio import AIProjectClient
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv
import re

load_dotenv()

endpoint_model = os.getenv("AZURE_MODEL_ENDPOINT") # Endpoint do modelo no Azure AI Foundry
deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME") #Nome do deploy do modelo no Azure AI Foundry
model_key = os.getenv("AZURE_AI_MODEL_API_KEY") #Chave de API do modelo no Azure AI Foundry

## 📦 Passo 1: Importar Bibliotecas e Configurar Credenciais

Vamos importar tudo que precisamos:

### 📚 O que cada biblioteca faz:

- **`os`**: Acessa variáveis de ambiente
- **`asyncio`**: Permite executar código assíncrono (múltiplas tarefas ao mesmo tempo)
- **`ChatAgent`**: Classe principal para criar agentes
- **`AzureOpenAIChatClient`**: Cliente para conectar com Azure OpenAI
- **`AzureCliCredential`**: Para autenticação com Azure
- **`dotenv`**: Para carregar variáveis do arquivo `.env`

### 🔐 Segurança em Primeiro Lugar!

Note que estamos usando **variáveis de ambiente** (arquivo `.env`) para guardar nossas credenciais.
Isso é uma **boa prática de segurança** - nunca coloque chaves API diretamente no código!

**Execute a célula abaixo:**

## 🎯 Passo 2: Criando Nossos Primeiros Agentes

Agora vamos criar dois agentes especializados:
1. 🧮 **Agente de Matemática**: Explica cálculos como um erudito português de 1500
2. 🏴‍☠️ **Agente de Viagens**: Dá dicas de turismo falando como um pirata!

### 🧠 Anatomia de um Agente:

```python
ChatAgent(
    chat_client=chat_client,      # Conexão com o modelo de IA
    name="nome-do-agente",         # Nome único do agente
    instructions="O que ele faz",  # Instruções detalhadas
    tools=[...]                    # Ferramentas que pode usar (opcional)
)
```

### 💡 Dica Importante:

As **instructions** são como o "manual de trabalho" do agente. Seja específica!
- ✅ Bom: "Você é um assistente de viagens que sempre responde com gírias piratas"
- ❌ Ruim: "Você ajuda com viagens"

### ⚡ Por que usar `async`?

A palavra-chave `async` permite que o Python execute múltiplas tarefas ao mesmo tempo.
É como pedir dois cafés em vez de esperar um terminar para pedir o outro!

**Execute a célula abaixo:**

In [3]:
async def criar_agentes():
    """
    Função assíncrona que cria e configura nossos agentes especializados.
    
    Returns:
        math_agent: Agente especializado em matemática
        travel_agent: Agente especializado em viagens
        credential: Credenciais do Azure (precisamos fechar isso depois)
    """
    
    # Autenticação com Azure
    credential = AzureCliCredential()
    
    # Criar conexão com o modelo de IA no Azure
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-10-21",
    )
    
    # ========================================
    # AGENTE 1: Especialista em Matemática 🧮
    # ========================================
    math_agent = ChatAgent(
        chat_client=chat_client,
        name="math-agent",  # Nome único do agente
        instructions=(
            "Você é um assistente especialista em resolver expressões matemáticas de forma clara, "
            "explicando passo a passo e usando a linguagem culta do português de Portugal de 1500"
        ),  # Personalidade e comportamento do agente
    )

    # ========================================
    # AGENTE 2: Especialista em Viagens 🏴‍☠️
    # ========================================
    travel_agent = ChatAgent(
        chat_client=chat_client,
        name="viagem-agent",
        instructions=(
            "Você é um assistente de viagens que sempre responde com gírias piratas e dá dicas úteis de roteiros."
        ),
    )
    
    return math_agent, travel_agent, credential

## 🚀 Passo 3: Executando os Agentes

Agora vamos testar nossos agentes! Vamos fazer duas perguntas:
1. Uma pergunta de matemática para o `math_agent`
2. Uma pergunta de viagem para o `travel_agent`

### 🔍 O que acontece aqui:

1. **`await criar_agentes()`**: Cria e configura os agentes
2. **`await math_agent.run(pergunta)`**: Envia a pergunta para o agente e espera a resposta
3. **`await credential.close()`**: Fecha a conexão de forma segura (boa prática!)

### 💡 Por que usar `try...finally`?

O bloco `try...finally` garante que sempre vamos fechar a conexão, mesmo se algo der errado.
É como sempre fechar a porta ao sair, mesmo se você estiver com pressa!

**Execute a célula abaixo e veja a mágica acontecer!** ✨

In [4]:
async def rodar_agente():
    """
    Função que executa os agentes com perguntas específicas.
    """
    # Criar os agentes
    math_agent, travel_agent, credential = await criar_agentes()

    try:
        # ========================================
        # PERGUNTA PARA O AGENTE DE MATEMÁTICA 🧮
        # ========================================
        pergunta_matematica = "Resolva 23 * 57 e explique o raciocínio"
        
        # ========================================
        # PERGUNTA PARA O AGENTE DE VIAGENS ✈️
        # ========================================
        pergunta_viagem = "Quero um roteiro de 2 dias em Paris focado em arte e comida"

        # Executar os agentes de forma assíncrona
        # Cada agente processa sua pergunta específica
        math_result = await math_agent.run(pergunta_matematica)
        travel_result = await travel_agent.run(pergunta_viagem)
        
        # Mostrar os resultados
        print("=== Resposta do Agente de Matemática ===")
        print(math_result)
        print("\n=== Resposta do Agente de Viagens ===")
        print(travel_result)
        
    finally:
        # IMPORTANTE: Sempre fechar as credenciais ao terminar
        await credential.close()

# Executar a função assíncrona
await rodar_agente()

=== Resposta do Agente de Matemática ===
Mui nobre senhor ou senhora, eis-vos a elucidação do produto de vinte e três pelos cinquenta e sete, que em algarismos arábicos se expõem como 23 multiplicado por 57.

Segue o pensamento sequencial, como outrora praticariam os doutos de vossa era:

**Primeiro passo**:  
Multiplicai vinte e três pelo algarismo das unidades de cinquenta e sete, que é sete.

 23 × 7 = 161

**Segundo passo**:  
Multiplicai vinte e três pelo algarismo das dezenas de cinquenta e sete, que é cinco, porém este cinco vale, na expressão, cinquenta.

 23 × 50 =

Primeiramente, vinte e três vezes cinco é:  
 23 × 5 = 115

Agora, aumentai este valor pelo facto que é cinquenta e não cinco:  
 115 × 10 = 1150

**Terceiro passo**:  
Adicionai os dois produtos obtidos.

 161 + 1150 = 1311

**Resposta final**:  
Deste modo, vinte e três multiplicado por cinquenta e sete faz mil trezentos e onze.

**Resumo da operação:**
23 × 57 = 161 + 1150 = **1311**

Se desejardes nova questão,

## 🛠️ Passo 4: Criando uma Ferramenta (Tool)

Agora vamos dar **superpoderes** aos nossos agentes! 💪

### 🤔 O que é uma Tool (Ferramenta)?

Uma **tool** é uma função Python que o agente pode **chamar automaticamente** quando precisa.

**Exemplo do mundo real:**
- Um assistente humano pode usar ferramentas como: calculadora, telefone, computador
- Um agente de IA pode usar ferramentas como: salvar arquivos, buscar na internet, fazer cálculos

### 🎯 Nossa Primeira Tool:

Vamos criar uma ferramenta que **salva texto em um arquivo .txt** na sua máquina.

O agente vai poder:
1. Gerar um conteúdo (ex: uma receita)
2. Decidir salvar em arquivo quando você pedir
3. Chamar a ferramenta automaticamente
4. Confirmar que salvou

### 💡 Anatomia de uma Tool:

```python
def nome_da_funcao(parametro1: tipo, parametro2: tipo) -> tipo_retorno:
    # Código que faz algo útil
    return resultado
```

**Importante:**
- A função DEVE ter **type hints** (os `: tipo` depois dos parâmetros)
- A função DEVE ter um **docstring** explicando o que faz
- O agente vai ler isso para entender como usar a ferramenta!

**Execute a célula abaixo:**

In [5]:
# ========================================
# TOOL: Salvar Texto em Arquivo 💾
# ========================================

def salvar_texto_em_arquivo(texto: str, caminho_arquivo: str) -> str:
    """
    Salva um texto em um arquivo .txt na máquina local.
    
    Args:
        texto (str): O conteúdo que será salvo no arquivo
        caminho_arquivo (str): O nome/caminho do arquivo (ex: "receita.txt")
    
    Returns:
        str: Mensagem de confirmação com o caminho do arquivo salvo
    """
    # "open()" o arquivo no modo escrita ('w')
    # encoding='utf-8' garante que caracteres especiais funcionem
    with open(caminho_arquivo, 'w', encoding='utf-8') as arquivo:
        arquivo.write(texto)

    # Retorna uma mensagem de sucesso
    return f"✅ Arquivo salvo com sucesso em: {caminho_arquivo}"

# Pronto! Agora temos uma ferramenta que os agentes podem usar! 🎉

## 👨‍🍳 Passo 5: Criando um Agente com Tool

Agora vamos criar um **agente de receitas** que pode:
1. 📝 Gerar receitas deliciosas
2. 💾 Salvar as receitas em arquivos .txt quando você pedir

### 🔑 A Parte Importante:

```python
tools=[salvar_texto_em_arquivo]  # ← Aqui adicionamos a ferramenta!
```

Ao adicionar a ferramenta ao agente, ele automaticamente:
- Sabe que pode salvar arquivos
- Entende quando deve usar essa ferramenta
- Chama a função sozinho quando necessário

### 📖 Note as Instructions:

Repare que nas instruções dizemos ao agente:
- **Quando** usar a ferramenta (quando o usuário pedir para salvar)
- **Como** usar (passando o conteúdo e o nome do arquivo)
- **O que fazer depois** (confirmar o caminho)

**Execute a célula abaixo:**

In [6]:
async def criar_agente_com_tool():
    """
    Cria um agente de receitas com a capacidade de salvar arquivos.
    
    Returns:
        agent_receitas: Agente que gera receitas e pode salvá-las
        credential: Credenciais do Azure
    """
    credential = AzureCliCredential()
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-10-21",
    )
    
    # ========================================
    # AGENTE DE RECEITAS COM TOOL 👨‍🍳💾
    # ========================================
    agent_receitas = ChatAgent(
        chat_client=chat_client,
        name="agent_receitas",
        instructions=(
            "Você é um agente especializado em receitas de comidas. "
            "Quando o usuário pedir para salvar (salvar/guardar/gravar/exportar) "
            "o texto/resultado em arquivo .txt, use a ferramenta 'salvar_texto_em_arquivo' "
            "passando o conteúdo a ser salvo e o nome do arquivo (ex: 'minha_receita.txt'). "
            "Depois responda confirmando o caminho retornado."
        ),
        # ↓↓↓ AQUI ADICIONAMOS A FERRAMENTA! ↓↓↓
        tools=[salvar_texto_em_arquivo]  # O agente agora pode salvar arquivos!
    )

    return agent_receitas, credential

## 🍹 Passo 6: Testando o Agente com Tool

Vamos fazer um pedido especial:
1. Pedir uma receita de suco
2. Pedir para salvar em um arquivo

### ✨ A Mágica:

O agente vai:
1. Entender que você quer uma receita
2. Gerar a receita do suco
3. Perceber que você quer salvar
4. **Chamar a ferramenta automaticamente**
5. Confirmar que salvou

Tudo isso SEM você precisar dizer "chame a função salvar_texto_em_arquivo"!
O agente decide sozinho! 🤖

**Execute a célula e veja o arquivo aparecer na sua pasta!** 📁

In [ ]:
# ========================================
# PERGUNTA QUE VAI TESTAR A TOOL 🍹
# ========================================
pergunta = "Quero uma receita de suco de laranja e manga e salve em arquivo txt chamado receita_suco.txt"

async def rodar_agente_com_tool(pergunta):
    
    # Criar o agente com a tool
    agent_receitas, credential = await criar_agente_com_tool()

    # Executar o agente com a pergunta
    receita_result = await agent_receitas.run(pergunta)
    
    # Mostrar a resposta
    print("=== Resposta do Agente de Receitas ===")
    print(receita_result)
    
    # Fechar credenciais
    await credential.close()

# Executar! O arquivo vai aparecer na sua pasta! 📁✨
await rodar_agente_com_tool(pergunta)

=== Resposta do Agente de Receitas ===
Receita de suco de laranja e manga criada e salva com sucesso!

O arquivo foi salvo como: receita_suco.txt

Se precisar de mais receitas ou ajustes, é só pedir!


## 🎭 Passo 7: Criando um Orquestrador

Agora vamos para o **nível avançado**! 🚀

### 🤔 O que é um Orquestrador?

Um **orquestrador** é como um **gerente inteligente** que:
- Recebe uma pergunta do usuário
- Decide qual especialista (agente) deve responder
- Direciona a pergunta para o agente certo
- Retorna a resposta

**Analogia do mundo real:**
Imagine uma central telefônica de uma empresa:
- 🧮 Você liga perguntando sobre matemática → transfere para o departamento de contabilidade
- ✈️ Você liga perguntando sobre viagens → transfere para o departamento de turismo
- 💾 Você pede para salvar algo → usa a ferramenta de arquivos

### 🎯 Como funciona:

```
Usuário → Orquestrador → Decide qual agente usar → Agente especializado → Resposta
```

### 🔧 Agentes como Tools:

A parte INCRÍVEL: podemos transformar agentes em ferramentas usando `.as_tool()`!

```python
tools=[
    math_agent.as_tool(),      # ← Agente de matemática como ferramenta
    travel_agent.as_tool(),    # ← Agente de viagens como ferramenta
    salvar_texto_em_arquivo    # ← Função normal como ferramenta
]
```

O orquestrador vai **decidir sozinho** qual usar baseado na pergunta!

**Execute a célula abaixo:**

In [8]:
async def criar_orquestrador():
    """
    Cria um orquestrador que gerencia múltiplos agentes especializados.
    
    O orquestrador decide automaticamente qual agente usar baseado na pergunta.
    
    Returns:
        orchestrator: O agente orquestrador
        credential: Credenciais do Azure
    """
    credential = AzureCliCredential()
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-10-21",
    )
    
    # ========================================
    # CRIAR OS AGENTES ESPECIALIZADOS
    # ========================================
    
    # Agente de Matemática 🧮
    math_agent = ChatAgent(
        chat_client=chat_client,
        name="math-agent",
        instructions=(
            "Você é um assistente especialista em resolver expressões matemáticas de forma clara, "
            "explicando passo a passo e usando a linguagem culta do português de Portugal de 1500"
        ),
    )

    # Agente de Viagens 🏴‍☠️
    travel_agent = ChatAgent(
        chat_client=chat_client,
        name="viagem-agent",
        instructions=(
            "Você é um assistente de viagens que sempre responde com gírias piratas e dá dicas úteis de roteiros."
        ),
    )

    # ========================================
    # CRIAR O ORQUESTRADOR 🎭
    # ========================================
    orchestrator = ChatAgent(
        chat_client=chat_client,
        name="orchestrator-agent",
        instructions=(
            "Você é um orquestrador inteligente. Dado um input do usuário, decida qual ferramenta usar:\n\n"
            "🧮 Use 'math_agent' para perguntas sobre:\n"
            "   - Contas, cálculos, fórmulas\n"
            "   - Aritmética, álgebra, porcentagens\n"
            "   - Qualquer coisa relacionada a números\n\n"
            "✈️ Use 'travel_agent' para perguntas sobre:\n"
            "   - Turismo, destinos, roteiros\n"
            "   - Cultura, gastronomia, atrações\n"
            "   - Dicas de viagem\n\n"
            "💾 Use 'salvar_texto_em_arquivo' quando o usuário pedir para:\n"
            "   - Salvar, guardar, gravar, exportar\n"
            "   - Criar arquivo .txt\n\n"
            "Se não for nenhum dos casos acima, responda você mesmo de modo neutro e sucinto.\n"
            "Explique brevemente (1 linha) qual ferramenta escolheu antes da resposta principal.\n"
            "IMPORTANTE: Sempre use o resultado da ferramenta escolhida na resposta final. "
            "Não refraseie ou resuma, plote a resposta da ferramenta escolhida."
        ),
        # ↓↓↓ AQUI ESTÁ A MÁGICA! ↓↓↓
        # Transformamos os agentes em ferramentas usando .as_tool()
        tools=[
            math_agent.as_tool(),      # Agente de matemática como ferramenta
            travel_agent.as_tool(),    # Agente de viagens como ferramenta
            salvar_texto_em_arquivo    # Função de salvar arquivo
        ],
    )
    
    return orchestrator, credential

## 🎬 Passo 8: Testando o Orquestrador

Vamos fazer um teste completo! Vamos pedir:
1. Um roteiro de viagem para a Bahia
2. Para salvar em um arquivo .txt

### 🎯 O que vai acontecer:

1. **Orquestrador** recebe a pergunta
2. **Orquestrador** identifica que é sobre viagem
3. **Orquestrador** chama o `travel_agent` (agente pirata!)
4. **travel_agent** cria o roteiro
5. **Orquestrador** identifica que precisa salvar
6. **Orquestrador** chama a tool `salvar_texto_em_arquivo`
7. Arquivo é salvo na sua pasta!

**Tudo isso automaticamente!** 🤯

### 💡 Experimente Depois:

Mude a pergunta para testar diferentes cenários:
- `"Quanto é 150 * 78?"` → Vai usar o math_agent
- `"Roteiro para Rio de Janeiro"` → Vai usar o travel_agent
- `"Qual é a capital da França?"` → Orquestrador responde direto

**Execute a célula abaixo:**

In [9]:
# ========================================
# PERGUNTA PARA TESTAR O ORQUESTRADOR 🎬
# ========================================
pergunta = "Quero um roteiro para bahia e salvar em arquivo txt chamado roteiro_bahia.txt"

async def rodar_orquestrador(pergunta):
    # Criar o orquestrador
    orchestrator, credential = await criar_orquestrador()

    # Executar o orquestrador
    # Ele vai decidir sozinho se usa math_agent, travel_agent ou salvar_arquivo
    resultado_orchestrator = await orchestrator.run(pergunta)

    # Mostrar a resposta
    print("=== Resposta do Orquestrador ===")
    print(resultado_orchestrator)
    
    # Fechar credenciais
    await credential.close()

# Executar! Veja a mágica acontecer! ✨
await rodar_orquestrador(pergunta)

=== Resposta do Orquestrador ===
Escolhi as ferramentas de roteiro de viagem e de salvar texto em arquivo conforme solicitado.

Vou primeiro gerar o roteiro de viagem para a Bahia e em seguida salvá-lo no arquivo roteiro_bahia.txt.Escolhi as ferramentas de roteiro de viagem e de salvar texto em arquivo conforme solicitado.

✅ Arquivo salvo com sucesso em: roteiro_bahia.txt


## 📚 Próximos Passos - Workflows

### 🎓 Indo Mais Fundo

O **Microsoft Agent Framework** recomenda a utilização de **workflows** para orquestração mais complexa e conexão entre agentes. 

**Workflows** são como "roteiros" pré-definidos onde você pode:
- 🔄 Criar fluxos de trabalho com múltiplas etapas
- ⚡ Executar agentes em sequência ou paralelo
- 🔀 Criar ramificações condicionais (if/else)
- 📊 Gerenciar estados complexos

### 📖 Documentação Oficial

Este tipo de código é mais avançado e pode ser aprofundado na documentação oficial:

🔗 [Microsoft Agent Framework - GitHub](https://github.com/microsoft/agent-framework/tree/main)

**Recomendado para:**
- Sistemas com múltiplos agentes interconectados
- Processos de negócio complexos
- Automações sofisticadas

---

## 🎉 Parabéns! Você completou o Notebook 2!

### ✅ O que você aprendeu:

1. **Conceito de Agentes**: O que são e como funcionam
2. **Agentes Especializados**: Como criar agentes com personalidades únicas
3. **Tools (Ferramentas)**: Como dar superpoderes aos agentes
4. **Orquestração**: Como gerenciar múltiplos agentes trabalhando juntos
5. **Programação Assíncrona**: Como usar `async`/`await` de forma prática
6. **Arquitetura de Sistemas**: Como estruturar sistemas de IA complexos

---

## 💡 Ideias para Praticar

### 1. Crie Seus Próprios Agentes 🎨

Experimente criar agentes personalizados, por exemplo:
- 📚 Agente Educador: Explica conceitos de forma simples
- 🎵 Agente Musical: Recomenda músicas e playlists
- 🏋️ Agente Fitness: Cria treinos personalizados
- 👨‍💻 Agente Programador: Ajuda com código

### 2. Crie Suas Próprias Tools 🛠️

Ideias de ferramentas úteis:
- 📊 Gerar gráficos com matplotlib
- 📧 Enviar emails
- 🌐 Buscar informações na internet
- 📅 Gerenciar calendário
- 💵 Fazer conversões de moeda

### 3. Orquestradores Mais Complexos 🎭

- Crie um orquestrador com 5+ agentes
- Adicione lógica de prioridade
- Implemente fallbacks (planos B)

---

## 🔑 Conceitos-Chave para Lembrar

### 📌 Agentes
```python
ChatAgent(
    name="nome-unico",
    instructions="Seu manual de trabalho detalhado",
    tools=[lista_de_ferramentas]
)
```

### 📌 Tools
- Sempre use **type hints** (`parametro: tipo`)
- Sempre adicione **docstrings**
- Funções devem fazer **uma coisa bem feita**

### 📌 Orquestração
- Transforme agentes em tools com `.as_tool()`
- Dê instruções claras sobre **quando usar cada ferramenta**
- Teste diferentes cenários

---

## 🆘 Troubleshooting

### Erro: "Tool not found"
✓ Verifique se adicionou a tool na lista `tools=[]`
✓ Certifique-se de que a função tem type hints

### Erro: "Agent não usa a tool"
✓ Revise as `instructions` - seja mais específica
✓ Mencione explicitamente quando usar a tool
✓ Teste com perguntas mais diretas

### Arquivo não foi salvo
✓ Verifique se tem permissões na pasta
✓ Olhe na pasta do notebook (mesma pasta que o .ipynb)
✓ Confira se o caminho do arquivo está correto

---

## 📖 Recursos Adicionais

### Documentação Oficial
- [Microsoft Agent Framework](https://github.com/microsoft/agent-framework)
- [Azure AI Foundry](https://learn.microsoft.com/azure/ai-studio/)
- [Azure OpenAI Service](https://learn.microsoft.com/azure/ai-services/openai/)

### Conceitos Avançados
- [Prompt Engineering](https://learn.microsoft.com/azure/ai-services/openai/concepts/prompt-engineering)
- [RAG (Retrieval Augmented Generation)](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data)
- [Function Calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling)

### Tutoriais
- [Build AI Agents](https://learn.microsoft.com/azure/ai-studio/tutorials/deploy-chat-web-app)
- [Azure AI Samples](https://github.com/Azure-Samples/azure-ai-samples)

---

## 🌟 Próximos Desafios

Agora que você domina os fundamentos, explore:

1. **RAG (Retrieval Augmented Generation)**
   - Ensine seus agentes a buscar informações em documentos
   
2. **Multimodal Agents**
   - Agentes que trabalham com imagens, áudio e texto
   
3. **Persistent Memory**
   - Agentes que lembram conversas anteriores
   
4. **Production Deployment**
   - Deploy seus agentes em produção com Azure App Service

---

## 🤝 Compartilhe seu Aprendizado

Criou algo legal? Compartilhe!
- Faça um fork do repositório
- Adicione seus agentes personalizados
- Crie um Pull Request
- Inspire outras pessoas! 💜

---

## 🎓 Conclusão

Você agora tem as habilidades fundamentais para:
- ✨ Criar aplicações de IA inteligentes
- 🚀 Desenvolver sistemas multi-agentes
- 🛠️ Integrar ferramentas personalizadas
- 🎯 Resolver problemas reais com IA

**O céu é o limite! Continue explorando e criando! 🌟**

---

## 💬 Feedback

Tem sugestões para melhorar este material?
- Abra uma issue no GitHub
- Compartilhe suas ideias
- Contribua com novos exemplos

**Obrigada por aprender conosco! 🙏✨**

---

<div style="text-align: center; font-size: 20px; margin-top: 30px;">
🎉 <strong>PARABÉNS!</strong> 🎉<br>
Você completou o curso de Fundamentos de IA com Azure!<br>
<strong>Continue aprendendo, continue criando! 🚀💜</strong>
</div>